In [2]:
import pandas as pd
import numpy as np
import json

csv_paths = ["data/annotations/ZZ_1.csv", "data/annotations/ZZ_2.csv", "data/annotations/ZZ_3.csv"]
for csv_path in csv_paths:
    marker = csv_path.split("/")[-1].split(".")[0]

    df = pd.read_csv(csv_path)
    # modify the column name
    for col in df.columns:
        df = df.rename(columns={col: col.strip().strip("\\n")})

    # replace nan with ""
    df = df.replace(np.nan, "")

    video = None
    video_list = []

    for row in df.iterrows():
        row = row[1]
        if row["视频序号"]:
            if video:
                video_list.append(video)
            video = {}
            video["video_id"] = marker + "_" + str(int(row["视频序号"]))
            video["video_url"] = row["视频_url"]
            video["video_duration"] = row["视频长度（min）"]
            video["video_type"] = row["视频类型"]
            video["qa_list"] = []
        qa = {
            "question": row["问题"],
            "answer": row["答案"],
            "question_type": row["问题类型"],
            "knowledge": row["若涉及通用知识推理，则写出推理步骤（知识点）"],
            "reasoning": row["依据（解题思路）"],
        }
        video["qa_list"].append(qa)

    video_list.append(video)

    print(f"Detected {len(video_list)} videos from {marker}")

    # save the video_list to a json file
    with open(f"data/annotations/{marker}.json", "w") as f:
        json.dump(video_list, f, ensure_ascii=False, indent=4)

Detected 100 videos from ZZ_1
Detected 100 videos from ZZ_2
Detected 100 videos from ZZ_3


In [5]:
import os
import json

processing_config = json.load(open("configs/processing_config.json"))
log_dir = processing_config["log_dir"]
os.makedirs(log_dir, exist_ok=True)

count = 0
annotaions_paths = ["data/annotations/ZZ_1.json", "data/annotations/ZZ_2.json", "data/annotations/ZZ_3.json"]

for annotaions_path in annotaions_paths:
    marker = annotaions_path.split("/")[-1].split(".")[0]
    with open(annotaions_path, "r") as f:
        data = json.load(f)

    for video in data:
        url = video["video_url"]
        if not url.startswith("https://www.youtube.com/watch?v="):
            count += 1
            with open(
                os.path.join(log_dir, f"annotation_processing_error.log"), "a"
            ) as f:
                f.write(f"Error video url from {marker}: {url}" + "\n")

print(f"Total error video urls: {count}")

Total error video urls: 1


In [7]:
from utils.chat_api import generate_messages, parallel_get_response
from prompts import prompt_refine_qa_list
from utils.general import validate_and_fix_python_list
import json

annotaions_paths = ["data/annotations/ZZ_1.json", "data/annotations/ZZ_2.json", "data/annotations/ZZ_3.json"]

for annotaions_path in annotaions_paths:
    marker = annotaions_path.split("/")[-1].split(".")[0]
    with open(annotaions_path, "r") as f:
        data = json.load(f)

    inputs = []

    for video in data:
        qa_list = video["qa_list"]
        qa_list = [
            {"question": qa["question"], "answer": qa["answer"]} for qa in qa_list
        ]
        input = [
            {"type": "text", "content": prompt_refine_qa_list.format(qa_list=qa_list)}
        ]
        inputs.append(input)

    messages = [generate_messages(input) for input in inputs]
    model = "gpt-4o-2024-11-20"

    responses = parallel_get_response(model, messages)[0]

    for video, response in zip(data, responses):
        refined_qa_list = validate_and_fix_python_list(response)
        for qa, refined_qa in zip(video["qa_list"], refined_qa_list):
            print(f"Original question: {qa['question']}")
            print(f"Refined question: {refined_qa['question']}")
            print(f"Original answer: {qa['answer']}")
            print(f"Refined answer: {refined_qa['answer']}")
            qa["question"] = refined_qa["question"]
            qa["answer"] = refined_qa["answer"]
            print("-" * 100)
        print("=" * 100)

    with open(f"data/annotations/{marker}_refined.json", "w") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

Original question: Is it permitted to fliming in Trattor Zaza?
Refined question: Is filming permitted in Trattoria Zaza?
Original answer: Yes, it is.
Refined answer: Yes, it is.
----------------------------------------------------------------------------------------------------
Original question: Which transportation modes were used when traveling among these three cities?
Refined question: What modes of transportation were used when traveling between these three cities?
Original answer: Airplane, train, and bus.
Refined answer: Airplane, train, and bus.
----------------------------------------------------------------------------------------------------
Original question: If Lauren Colvin forgets to bring something during the trip, what will she do?
Refined question: If Lauren Colvin forgets to bring something during the trip, what will she do?
Original answer: She would directly order on Amazon for it.
Refined answer: She would order it directly on Amazon.
----------------------------